In [1]:
# Пример, потом сгенерирую свои параметры, смотрите, пожалуйста, ниже
# – простое число p - модуль эллиптической кривой;

p = 6277101735386680763835789423207666416083908700390324961279
F = GF(p)

# Sage: https://doc.sagemath.org/html/en/constructions/elliptic_curves.html
# – эллиптическая кривая E, задаваемая a,b from F
a = 21
b = 10

In [20]:
E = EllipticCurve(F, [0,0,0,a,b])

In [21]:
E

Elliptic Curve defined by y^2 = x^3 + 21*x + 10 over Finite Field of size 6277101735386680763835789423207666416083908700390324961279

In [22]:
# порядок группы точек эллиптическойф кривой Е
m = E.cardinality()

In [23]:
m.nbits()


192

In [ ]:
# Нужно сгенерировать кривую с большим простым порядом >= 254 бит <= 255 bit

In [45]:
#p = random_prime(2^255-1,False,2^254)
p = 30699232851861250083325455112151246498963906158005131330497024639069102717541

In [46]:
is_prime(p)

True

In [47]:
print(p)

30699232851861250083325455112151246498963906158005131330497024639069102717541


In [48]:
p.nbits()

255

In [63]:
def check_gost_255_parameters(E, p, q) -> bool:
    """
    Проверяет Е по требованиям ГОСТ для m = 255 бит 
    """
    m = E.cardinality()
    
    for t in range(1,32):
        if (p ^ t) % q == 1:
            return False
    
    if p == m:
        return False
    
    if E.j_invariant() in [0, 1728]:
        return False

    return True

In [69]:
# Можно пропустить, дальше будет взят результат подбора
# Теперь нам нужно подобрать параметр б, чтобы порядок кривой был большой, ради этого мы и задали большой простой р
F = GF(p)

# Sage: https://doc.sagemath.org/html/en/constructions/elliptic_curves.html
# – эллиптическая кривая E, задаваемая a,b from F
a = 21
b = 1

# сделаем 100 попыток
for i in range(1,100):
    b = i
    try:
        E = EllipticCurve(F, [0,0,0,a,b])
    except:
        continue
    # порядок группы точек эллиптическойф кривой Е
    m = E.cardinality()
    q = m

    if is_prime(m) and m.nbits() == 255 and check_gost_255_parameters(E, p, q):
        print('Success')
        break
else:
    print('bad')
    

Success


In [70]:
print(b)

91


In [71]:
m = E.cardinality()
q = m

if is_prime(m) and m.nbits() == 255 and check_gost_255_parameters(E, p, q):
    print('Success')
else:
    print('bad')

Success


In [98]:
# б = 91 оказалось подходящим по условия
# (Из условий Александра Романовича): порядки точек совпадают - простой порядок всей группы гарантирует это
# (ГОСТ) проверили требования к параметрам кривой в ГОСТ: check_gost_255_parameters
p = 30699232851861250083325455112151246498963906158005131330497024639069102717541
F = GF(p)
a = 21
b = 91
E = EllipticCurve(F, [0,0,0,a,b])
m = E.cardinality()
q = m
# Подходит любая ненулевая точка, тк мы сгенерирорвали так, что порядки всех точек равны
P = E.lift_x(1)
#
d = 8791436124389467141524128416394596
Q = d*P

In [9]:
E

Elliptic Curve defined by y^2 = x^3 + 21*x + 91 over Finite Field of size 30699232851861250083325455112151246498963906158005131330497024639069102717541

In [13]:
# Test
P = E.lift_x(1)
print(P)
print(2*P)
print(7*P)

(1 : 16576249172647006302783497141349991181894673534486872887450383999092641742937 : 1)
(14398755231403949154126098415433770481814929436940459827578250494430641097607 : 1903631450071985320187196060631873931150882524998305363972302806030518131230 : 1)
(3831251870054767149331716123926601145979175706373338315764134263650556193070 : 26929041663542165766876799643413977290770284469548127501197352035226743539232 : 1)


In [63]:
# 2.Формирование цифровой подписи
# Шаг 1
# hash(message) беру из прошлой лабы, тк мой Стрибог не интегрирован в блокнот
message_hash = '5b8c33beef3db4e41a5658a2a6f621de9e7d578b74c3eda16d117dce3e533'

In [64]:
print(bin(int('c',16))[2:])

1100


In [65]:
msg_hash_bin = map(lambda x: bin(int(x, 16))[2:], message_hash)

In [66]:
msg_hash_bin = list(map(lambda x: '0'*(4-len(x)) + x, msg_hash_bin))

In [67]:
msg_hash_bin = ''.join(msg_hash_bin)

In [68]:
msg_hash_bin

'0101101110001100001100111011111011101111001111011011010011100100000110100101011001011000101000101010011011110110001000011101111010011110011111010101011110001011011101001100001111101101101000010110110100010001011111011100111000111110010100110011'

In [40]:
# Напишем вспомогательную функцию:
def convert_hash_to_int(msg_hash:str) -> int:
    msg_hash_bin = map(lambda x: bin(int(x, 16))[2:], msg_hash)
    msg_hash_bin = list(map(lambda x: '0'*(4-len(x)) + x, msg_hash_bin))
    msg_hash_bin = ''.join(msg_hash_bin)
    return int(msg_hash_bin, 2)

In [71]:
# Шаг 2
#_a = int(msg_hash_bin, 2)
_a = convert_hash_to_int(message_hash)
_a

10109420153026071913927282020754067617368409882790886919067076269781017907

In [72]:
e = _a % q

In [73]:
if e == 0:
    e = 1

In [74]:
def compute_e(_a:int, q:int) -> int:
    e = _a % q
    return e if e != 0 else 1

In [76]:
e = compute_e(_a, q)
e

10109420153026071913927282020754067617368409882790886919067076269781017907

In [30]:
# Код для генерации, идём к следу пункту
r = 0
k = 0
while True:
    k = randint(1, q)
    C = k * P
    r = int(C[0]) % q
    if r != 0:
        break
print(r)
print(k)

14189083577683248705338699822309083472028665172171027963340404431292367084468
25364974838221291256710170366999887406724464100187569756527148590861962025651


In [97]:
r = 14189083577683248705338699822309083472028665172171027963340404431292367084468
k = 25364974838221291256710170366999887406724464100187569756527148590861962025651

In [78]:
s = (r*d + k*e) % q
s

14180222186159490648104699285312752785945763220296305490492321926222830351289

In [79]:
r_ = bin(r)[2:]
s_ = bin(s)[2:]
digital_signature = r_ + s_
digital_signature

'11111010111101011110010011000110010100010001110011100011101100010011000111100000101110111010101001111011010101110101000101110000011101111110000001011101100101001101000000001100111011001100010011010110111011011001111101000110101010100011110011011101101001111101011001101110001010100110000010100110101000001011100110011110001000011001110101011101101011011111111110101001000010011110011010101001011100010111000000110111100011101111100101001011101101101110010100110101100110100110001111000111111011001110111001'

In [99]:
# 3.Проверка ЦП
l = len(digital_signature) / 2
r = int(digital_signature[0:l], 2)
print(r)
s = int(digital_signature[l:], 2)
print(s)

14189083577683248705338699822309083472028665172171027963340404431292367084468
14180222186159490648104699285312752785945763220296305490492321926222830351289


In [100]:
# Шаг 2: вычислить хэш от сообщение - берём его из раннее вычисленного

In [101]:
# Шаг 3:
_a = convert_hash_to_int(message_hash)
_a

10109420153026071913927282020754067617368409882790886919067076269781017907

In [102]:
e = compute_e(_a, q)
e

10109420153026071913927282020754067617368409882790886919067076269781017907

In [103]:
# Шаг  4
modRing = IntegerModRing(q)
v = int(modRing(e)^(-1))
v

12573442438759871681318639583674257779453190654227207490550087087629621727427

In [104]:
print(e*v % q == 1)

True


In [105]:
# Шаг 5
z1 = int(modRing(s*v))
print(z1)
z2 = int(modRing(-1*r*v))
print(z2)

2388958791059351180544408501426030314817377899707863336142133522421550380284
27052272878704522276373726035313232179293446216760665224490816967509246703829


In [106]:
# Шаг 6:
C = z1*P + z2*Q
C

(14189083577683248705338699822309083472028665172171027963340404431292367084468 : 6320501109224377206438975229473014763890586457809139229047545248997237319691 : 1)

In [107]:
# Шаг 7:
R = int(modRing(C[0]))
print(R)
if R == r:
    print('signature is TRUE')
else:
    print('signature is FALSE')

14189083577683248705338699822309083472028665172171027963340404431292367084468
signature is TRUE


In [ ]:
# Ура!!! работает